In [5]:
from koebe.algorithms.tiling import *
from koebe.geometries.euclidean2 import PointE2

In [8]:
def midp(dart):
    u = dart.origin.point - PointE2.O
    v = dart.dest.point - PointE2.O
    vs = dart.splitVertices
    vs[0].point = PointE2.O + (u + v) / 2
    return vs[0].point

def chair_newverts(vDict):
    a = vDict["a"].point
    h = vDict["h"].point
    A = vDict["A"].point
    
    e0 = a - A
    e1 = h - A
    
    vDict["i"].point = A + 3 * e0 + e1
    vDict["j"].point = A + 2 * e0 + e1
    vDict["k"].point = A + e0 + e1
    vDict["l"].point = A + e0 + 2 * e1
    vDict["m"].point = A + e0 + 3 * e1

# Chair Tiling

In [27]:
########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
rules = TilingRules()
chair = rules.createPrototile("chair", [("A", PointE2(0,0)),
                                        ("B", PointE2(1,0)),
                                        ("C", PointE2(2,0)),
                                        ("D", PointE2(2,1)),
                                        ("E", PointE2(1,1)),
                                        ("F", PointE2(1,2)),
                                        ("G", PointE2(0,2)),
                                        ("H", PointE2(0,1))])

# Edges that need to be split
chair.addSplitEdgeRules(((("A","B"), ("a"), midp), (("B","C"), ("b"), midp), (("C","D"), ("c"), midp),
                        (("D","E"), ("d"), midp), (("E","F"), ("e"), midp), (("F","G"), ("f"), midp),
                        (("G","H"), ("g"), midp), (("H","A"), ("h"), midp)))

# New vertices to create
chair.addNewVertexRules(("i","j","k","l","m"))
chair.setNewVertexHandlerFn(chair_newverts)

# The subdivision subtiles: 
chair.addSubtile("chair", ("A", "a", "B", "j", "k", "l", "H", "h"))
chair.addSubtile("chair", ("G", "g", "H", "l", "m", "e", "F", "f"))
chair.addSubtile("chair", ("k", "j", "i", "d", "E", "e", "m", "l"))
chair.addSubtile("chair", ("C", "c", "D", "d", "i", "j", "B", "b"))

########################################
# Apply the rules
########################################

tiling = rules.generateTiling("chair", depth = 6)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
tile_vertex_count = len(tiling.verts)
print(tile_vertex_count)
original_tiling = tiling
#tiling = tiling.duplicate()


12545


In [28]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import E2Viewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def mvPt(p):
    return PointE2(p.x * 300 - 300, p.y * 300 - 300)

def pointSegmentE2For(dart):
    return SegmentE2(mvPt(dart.origin.point), mvPt(dart.dest.point))

viewer = E2Viewer(600, 600)

# Ucomment to view the triangulation vertices
# triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
# triSegs = [(SegmentE2For(e.aDart), triSegStyle)
#            for e in packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
# viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(pointSegmentE2For(e.aDart), edgeStyle)
           for e in tiling.edges]
viewer.addAll(edgeSegs)

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "SegmentE2", "endpoints": [[-290.625, -300.0], [-290.625, -295.3125]]…

In [5]:
starTriangulateAllFaces(tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
packing, _ = maximalPacking(
    tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(packing.verts)):
    packing.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

In [6]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 3
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in packing.verts])

# Ucomment to view the triangulation vertices
# triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
# triSegs = [(SegmentE2For(e.aDart), triSegStyle)
#            for e in packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
# viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

for vIdx in range(len(original_tiling.verts)):
    original_tiling.verts[vIdx].idx = vIdx
    original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Twisted pentagonal tiling

In [2]:
from koebe.algorithms.tiling import *
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
pent_rules = TilingRules()

pent = pent_rules.createPrototile("pent", tuple("ABCDE"))

# Edges that need to be split
pent.addSplitEdgeRules(((("A","B"), ("a", "b")), 
                        (("B","C"), ("c", "d")), 
                        (("C","D"), ("e", "f")),
                        (("D","E"), ("g", "h")), 
                        (("E","A"), ("i", "j"))))

# New vertices to create
pent.addNewVertexRules(("k"))

# The subdivision subtiles: 
pent.addSubtile("pent", tuple("Aabkj"))
pent.addSubtile("pent", tuple("Bcdkb"))
pent.addSubtile("pent", tuple("Cefkd"))
pent.addSubtile("pent", tuple("Dghkf"))
pent.addSubtile("pent", tuple("Eijkh"))

########################################
# Apply the rules
########################################

pent_tiling = pent_rules.generateTiling("pent", depth = 3)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
pent_tile_vertex_count = len(pent_tiling.verts)
original_pent_tiling = pent_tiling
pent_tiling = pent_tiling.duplicate()
#starTriangulateAllFaces(pent_tiling)


In [3]:
starTriangulateAllFaces(pent_tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
pent_packing, _ = maximalPacking(
    pent_tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(pent_packing.verts)):
    pent_packing.verts[vIdx].is_tile_vertex = vIdx < pent_tile_vertex_count

In [4]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 1
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_pent_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in pent_packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in pent_packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in pent_packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

# for vIdx in range(len(original_pent_tiling.verts)):
#     original_tiling.verts[vIdx].idx = vIdx
#     original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

In [5]:
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

tutteGraph = tutteEmbeddingE2(original_pent_tiling, False, True)

for vIdx in range(len(tutteGraph.verts)):
    tutteGraph.verts[vIdx].is_tile_vertex = vIdx < pent_tile_vertex_count

Duplicating DCEL...
done.
Reordering vertices...
done.
Laying out boundary...
done.
Computing graph laplacian...
Creating vertToIdx array...
Creating vertToDeg array...
Creating mat...
Done.
Returning coo_matrix with shape (256, 256)
done.
Computing Tutte embedding...
Computing P1...
Computing L1...
Computing B...
Computing L2...
Computing -inv(L2)
Computing nInvL2*B*P1
Concatenating P1 and P2
done.
Setting .data attributes...
done.


In [6]:
print(f"Vertex count: {len(tutteGraph.verts)}")
print(f"Vertex edges: {len(tutteGraph.edges)}")
print(f"Face count: {(len(tutteGraph.faces)-1)}")

Vertex count: 256
Vertex edges: 380
Face count: 125


In [7]:
from koebe.geometries.euclidean2 import SegmentE2
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
viewer = UnitScaleE2Sketch()

triSegStyle = makeStyle(stroke="#fece00", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), triSegStyle)
    for e in tutteGraph.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), edgeStyle)
    for e in tutteGraph.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
)

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[0.9989171113385246, 0.04652531219774665],…

# Twisted Hexagonal Tiling

In [602]:
########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
hex_rules = TilingRules()

hex = hex_rules.createPrototile("hex", ["A_A", "B", "C", "D", "E", "F"])

# Edges that need to be split
hex.addSplitEdgeRules(( 
                        (("A_A","B"), ("a1", "a2", "a3")), 
                        (("B","C"), ("b1", "b2", "b3")), 
                        (("C","D"), ("c1", "c2", "c3")),
                        (("D","E"), ("d1", "d2", "d3")), 
                        (("E","F"), ("e1", "e2", "e3")),
                        (("F","A_A"), ("f1", "f2", "f3"))
                      ))

# New vertices to create
hex.addNewVertexRules(("x"))

# The subdivision subtiles: 
hex.addSubtile("hex", ("A_A", "a1", "a2", "a3", "x", "f3"))
hex.addSubtile("hex", ("B", "b1", "b2", "b3", "x", "a3"))
hex.addSubtile("hex", ("C", "c1", "c2", "c3", "x", "b3"))
hex.addSubtile("hex", ("D", "d1", "d2", "d3", "x", "c3"))
hex.addSubtile("hex", ("E", "e1", "e2", "e3", "x", "d3"))
hex.addSubtile("hex", ("F", "f1", "f2", "f3", "x", "e3"))

########################################
# Apply the rules
########################################

hex_tiling = hex_rules.generateTiling("hex", depth = 4)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
hex_tile_vertex_count = len(hex_tiling.verts)
original_hex_tiling = hex_tiling
hex_tiling = hex_tiling.duplicate()
starTriangulateAllFaces(hex_tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
hex_packing, _ = maximalPacking(
    hex_tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(hex_packing.verts)):
    hex_packing.verts[vIdx].is_tile_vertex = vIdx < hex_tile_vertex_count

In [603]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 1
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_pent_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in hex_packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in hex_packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in hex_packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

# for vIdx in range(len(original_pent_tiling.verts)):
#     original_tiling.verts[vIdx].idx = vIdx
#     original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Twisted n-agonal tiling

In [643]:
########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
N = 9
ntile_rules = TilingRules()

ntile = ntile_rules.createPrototile("ntile", [f"V{i}" for i in range(N)])

# Edges that need to be split
ntile.addSplitEdgeRules([((f"V{i}",f"V{(i+1)%N}"), [f"v{i}_{j}" for j in range(1, N-2)])
                        for i in range(N)])

# New vertices to create
ntile.addNewVertexRules(("x"))

# The subdivision subtiles: 
for i in range(N):
    ntile.addSubtile("ntile", [f"V{i}"] + [f"v{i}_{j}" for j in range(1, N-2)] + ["x", f"v{(i+N-1)%N}_{N-3}"])

########################################
# Apply the rules
########################################

ntile_tiling = ntile_rules.generateTiling("ntile", depth = 4)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
ntile_tile_vertex_count = len(ntile_tiling.verts)
original_ntile_tiling = ntile_tiling
ntile_tiling = ntile_tiling.duplicate()
starTriangulateAllFaces(ntile_tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
ntile_packing, _ = maximalPacking(
    ntile_tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(ntile_packing.verts)):
    ntile_packing.verts[vIdx].is_tile_vertex = vIdx < ntile_tile_vertex_count

In [642]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 1
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_pent_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in ntile_packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in ntile_packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in ntile_packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

# for vIdx in range(len(original_pent_tiling.verts)):
#     original_tiling.verts[vIdx].idx = vIdx
#     original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Two-Prototile Experimental

In [10]:
from koebe.algorithms.tiling import *
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
rules = TilingRules()

t0 = rules.createPrototile("t0", tuple("AB"))
t1 = rules.createPrototile("t1", tuple("ABC"))
t2 = rules.createPrototile("t2", tuple("ABCDEF"))
t3 = rules.createPrototile("t3", tuple("ABCD"))

# Edges that need to be split
t1.addSplitEdgeRules((
    (("A", "B"), ("a1", "a2")),
    (("B", "C"), ("b")),
    (("C", "A"), ("c"))
))
t2.addSplitEdgeRules((
    (("A", "B"), ("a1", "a2")),
    (("B", "C"), ("b1", "b2")),
    (("C", "D"), ("c1", "c2")),
    (("D", "E"), ["d1"]),
    (("E", "F"), ("e1", "e2")),
    (("F", "A"), ["f1"]),
))
t3.addSplitEdgeRules((
    (("A", "B"), ["a1", "a2"]), 
    (("B", "C"), ["b1"]), 
    (("C", "D"), ["c1", "c2"]), 
    (("D", "A"), ["d1"])
))

# New vertices to create
t0.addNewVertexRules(("x"))
t2.addNewVertexRules(("x", "y"))
t3.addNewVertexRules(("x", "y"))

# The subdivision subtiles: 
t0.addSubtile("t1", ("A", "B", "x"))
t0.addSubtile("t1", ("B", "A", "x"))
#t0.addSubtile("t1", ("C", "A", "x"))

t1.addSubtile("t1", ("c", "b", "C"))
t1.addSubtile("t2", ("A", "a1", "a2", "B", "b", "c"))

t2.addSubtile("t3", ("f1", "x", "e2", "F"))
t2.addSubtile("t3", ("x", "y", "e1", "e2"))
t2.addSubtile("t3", ("y", "d1", "E", "e1"))
t2.addSubtile("t2", ("A", "a1", "a2", "B", "x", "f1"))
t2.addSubtile("t2", ("B", "b1", "b2", "C", "y", "x"))
t2.addSubtile("t2", ("C", "c1", "c2", "D", "d1", "y"))

t3.addSubtile("t3", ("A", "a1", "x", "d1"))
t3.addSubtile("t3", ("a1", "a2", "y", "x"))
t3.addSubtile("t3", ("a2", "B", "b1", "y"))
t3.addSubtile("t3", ("y", "b1", "C", "c1"))
t3.addSubtile("t3", ("x", "y", "c1", "c2"))
t3.addSubtile("t3", ("d1", "x", "c2", "D"))


########################################
# Apply the rules
########################################

print("Generating tiling...")
tiling = rules.generateTiling("t0", depth = 8)
print(len(tiling.verts))
print(len(tiling.darts))
print(len(tiling.faces))
tile_vertex_count = len(tiling.verts)

########################################
# Circle pack to get geometry. 
########################################

# print("Circle packing...")
# import koebe.algorithms.hypPacker
# koebe.algorithms.hypPacker._REPACK_DEBUG = True
# packing, _ = generateCirclePackingLayout(tiling)

Generating tiling...
116349
456648
111977


In [5]:
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2
tutteGraph = tutteEmbeddingE2(tiling, False, True)

for vIdx in range(len(tutteGraph.verts)):
    tutteGraph.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

Duplicating DCEL...
done.
Reordering vertices...
done.
Laying out boundary...
done.
Computing graph laplacian...
Creating vertToIdx array...
Creating vertToDeg array...
Creating mat...
Done.
Returning coo_matrix with shape (20121, 20121)
done.
Computing Tutte embedding...
Computing P1...
Computing L1...
Computing B...
Computing L2...
Computing -inv(L2)
Computing nInvL2*B*P1
Concatenating P1 and P2
done.
Setting .data attributes...
done.


NameError: name 'tile_vertex_count' is not defined

In [9]:
tile_vertex_count = 20121
for vIdx in range(len(tutteGraph.verts)):
    tutteGraph.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

from koebe.geometries.euclidean2 import SegmentE2
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
viewer = UnitScaleE2Sketch()

triSegStyle = makeStyle(stroke="#fece00", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), triSegStyle)
    for e in tutteGraph.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), edgeStyle)
    for e in tutteGraph.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
)

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[0.0009220693342046941, -3.973640849762822…

In [12]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

TilingViewer(packing).show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Some older stuff that may be useful for debugging so I'm not deleting it yet. 

In [499]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(400, 400)

# Uncomment to show super-tiles
level = 3
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163"]
# for i in range(1,len(original_tiling.faceLevels[level])):
#     for tile in collectFaces(original_tiling.faceLevels[level][i]):
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=colors[i%4]))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

for vIdx in range(len(original_tiling.verts)):
    original_tiling.verts[vIdx].idx = vIdx
    original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#000", "strokeW…

In [430]:
# Useful debugging checks: 

tile_vertex_count = len(tiling.verts)

dartEdgeError = False
dartTwinError = False
dartNextPrevError = False
dartFaceError = False
dartVertexError = False

for dart in tiling.darts:
    if dart != dart.edge.aDart and dart.twin != dart.edge.aDart:
        dartEdgeError = True
    if dart.twin.twin != dart:
        dartTwinError = True
    if dart.next.prev != dart:
        dartNextPrevError = True
    if dart not in dart.face.darts():
        dartFaceError = True
    if dart not in dart.origin.outDarts():
        dartVertexError = True

vertexDartError = False
for vertex in tiling.verts:
    for dart in vertex.outDarts():
        if vertex != dart.origin:
            print(tiling.verts.index(vertex))
            vertexDartError = True
            
print(f"Detected a dartEdgeError: {dartEdgeError}")
print(f"Detected a dartTwinError: {dartTwinError}")
print(f"Detected a dartNextPrevError: {dartNextPrevError}")
print(f"Detected a dartFaceError: {dartFaceError}")
print(f"Detected a dartVertexError: {dartVertexError}")
print(f"Detected a vertexDartError: {vertexDartError}")

Detected a dartEdgeError: False
Detected a dartTwinError: False
Detected a dartNextPrevError: False
Detected a dartFaceError: False
Detected a dartVertexError: False
Detected a vertexDartError: False


In [438]:
from koebe.algorithms.hypPacker import *
packing, _ = maximalPacking(
    tiling, 
    num_passes=1000
)

for vIdx in range(len(packing.verts)):
    packing.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

In [439]:
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(400, 400)
#viewer.addAll([v.data for v in packing.verts])
edgeSegs = [SegmentE2For(e.aDart)
           for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def showDartAndNext(viewer, dart):
    viewer.add(SegmentE2For(dart),      makeStyle(stroke="#0f0", strokeWeight=3.0))
    viewer.add(SegmentE2For(dart.next), makeStyle(stroke="#f00", strokeWeight=3.0))

def showDartAndTwin(viewer, dart):
    viewer.add(SegmentE2For(dart),      makeStyle(stroke="#0f0", strokeWeight=6.0))
    viewer.add(SegmentE2For(dart.twin), makeStyle(stroke="#f00", strokeWeight=3.0))

def showOutDarts(viewer, vertex):
    for dart in vertex.outDarts(): viewer.add(SegmentE2For(dart), makeStyle(stroke="#f00", strokeWeight=6.0))
    viewer.add(PointE2For(vertex), makeStyle(fill="#0f0"))
    
def showInDarts(viewer, vertex):
    for dart in vertex.inDarts(): viewer.add(SegmentE2For(dart), makeStyle(stroke="#f00", strokeWeight=6.0))
    viewer.add(PointE2For(vertex), makeStyle(fill="#0f0"))

# showDartAndTwin(viewer, packing.darts[82])
# showDartAndTwin(viewer, packing.darts[85])

#i += 1
#showInDarts(viewer, packing.verts[i])
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#000", "strokeW…

In [160]:
packing.verts[0].data

CircleH2(center=PointH2(coord=ExtendedComplex(z=(-0.11851176705023778-0.9929526479498557j), w=(1+0j), _complex=(-0.11851176705023778-0.9929526479498557j))), xRadius=-0.07986492414663876)

In [7]:
from koebe.geometries.euclidean2 import PointE2, SegmentE2
viewer = PoincareDiskViewer(600, 600)
edgeSegs = [SegmentE2(PointE2(e.aDart.origin.data.center.coord.real, e.aDart.origin.data.center.coord.imag), 
                      PointE2(e.aDart.dest.data.center.coord.real, e.aDart.dest.data.center.coord.imag))
           for e in packing.edges]
viewer.addAll(edgeSegs)
blue = makeStyle(stroke = "#79f", strokeWeight=1.0)
red = makeStyle(stroke = "#f97", strokeWeight=1.0)
green = makeStyle(stroke = "#7f9", strokeWeight=1.0)
gray = makeStyle(stroke = "#999", strokeWeight=1.0)
for i in range(len(edgeSegs)):
    e = edgeSegs[i]
    if tiling.edges[i].level % 4 == 0:
        viewer.setStyle(e, blue)
    elif tiling.edges[i].level % 4 == 1:
        viewer.setStyle(e, red)
    elif tiling.edges[i].level % 4 == 2:
        viewer.setStyle(e, green)
    else:
        viewer.setStyle(e, gray)
viewer.show()

<IPython.core.display.Javascript object>

AttributeError: 'Edge' object has no attribute 'level'

In [60]:
tiling.__class__

__main__.Tiling

In [5]:
tiling = Tiling.generateCycle(vdata = ["A","B","C","D","E","F","G","H"])

In [7]:
tiling.darts

In [451]:
class Foo:
    pass

x = Foo()

x.myAttribute = "now it does"

In [452]:
x.myAttribute

'now it does'